# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

Note: Make sure that you have PostgreSQL running.

## Imports

In [496]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Generate Test Data

In [497]:
# Create two blobs of data
prev_month_blobs = h.generate_random_polygons(n=10000)
curr_month_blobs = h.generate_random_polygons(n=10000)

Function `generate_random_polygons` executed in 7.7969 sec, CPU: 4.30%, Memory: 63.72MB
Function `generate_random_polygons` executed in 7.8217 sec, CPU: 25.80%, Memory: 75.25MB


In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
8507,"POLYGON ((-98.16996 26.92231, -98.16996 26.922...",e8ffb175-5a75-44ab-abb0-b1b251411dcb,9uf1nj55
9127,"POLYGON ((-98.24604 27.17870, -98.24604 27.179...",699be616-0f16-4aec-9658-8e1fb7c995db,9uf4s7g8
1665,"POLYGON ((-98.25891 27.30433, -98.25891 27.305...",4626d0ee-a7c1-4f8f-8f0f-0c2b351e59c2,9uf5k4dn
3025,"POLYGON ((-97.90124 27.65322, -97.90124 27.653...",7849f3ec-fb7d-4caa-bae2-517c089283a5,9ufmk4nt
1518,"POLYGON ((-98.21733 27.67615, -98.21733 27.676...",4708cb3c-6db9-4f7f-bb12-1491c09fcbe1,9ufjmn2d


In [ ]:
curr_month_blobs.head()

,geometry,id,geohash
7825,"POLYGON ((-81.83501 27.05556, -81.83501 27.056...",73aa18b8-191d-4580-afb5-a9ef667f4fb5,dhvccv3g
8049,"POLYGON ((-81.79777 27.53440, -81.79777 27.534...",0cdcd373-c056-43b0-9940-8fec66004f2a,dhvudskz
862,"POLYGON ((-82.18730 27.70783, -82.18730 27.708...",fed6270b-3fe1-4992-bdfe-97d3b7b93096,dhvt9u1k
6064,"POLYGON ((-82.30218 27.77151, -82.30218 27.771...",d465a328-be4a-4cc4-8d8d-34f75ad99d88,dhvmzptt
3123,"POLYGON ((-82.39378 27.90654, -82.39378 27.907...",7329582d-5cd7-4477-a782-16b92c1c18ed,dhvqv06b


In [ ]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [ ]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
8507,"POLYGON ((-98.16996 26.92231, -98.16996 26.922...",e8ffb175-5a75-44ab-abb0-b1b251411dcb,9uf1nj55
9127,"POLYGON ((-98.24604 27.17870, -98.24604 27.179...",699be616-0f16-4aec-9658-8e1fb7c995db,9uf4s7g8
1665,"POLYGON ((-98.25891 27.30433, -98.25891 27.305...",4626d0ee-a7c1-4f8f-8f0f-0c2b351e59c2,9uf5k4dn
3025,"POLYGON ((-97.90124 27.65322, -97.90124 27.653...",7849f3ec-fb7d-4caa-bae2-517c089283a5,9ufmk4nt
1518,"POLYGON ((-98.21733 27.67615, -98.21733 27.676...",4708cb3c-6db9-4f7f-bb12-1491c09fcbe1,9ufjmn2d


## Non-optimized process

### 1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [ ]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [ ]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.0498 sec, CPU: 48.40%, Memory: 0.05MB
Function `convert_col_to_string` executed in 0.0494 sec, CPU: 34.10%, Memory: 7.47MB
                                               geometry  \
8507  POLYGON ((-98.169962 26.922313, -98.169962 26....   
9127  POLYGON ((-98.246043 27.178699, -98.246043 27....   
1665  POLYGON ((-98.258909 27.304334, -98.258909 27....   
3025  POLYGON ((-97.901237 27.653225, -97.901237 27....   
1518  POLYGON ((-98.217326 27.676151, -98.217326 27....   

                                        id   geohash  
8507  e8ffb175-5a75-44ab-abb0-b1b251411dcb  9uf1nj55  
9127  699be616-0f16-4aec-9658-8e1fb7c995db  9uf4s7g8  
1665  4626d0ee-a7c1-4f8f-8f0f-0c2b351e59c2  9uf5k4dn  
3025  7849f3ec-fb7d-4caa-bae2-517c089283a5  9ufmk4nt  
1518  4708cb3c-6db9-4f7f-bb12-1491c09fcbe1  9ufjmn2d  


/Users/sra/files/projects/matching_optimization/helpers.py:221: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))
/Users/sra/files/projects/matching_optimization/helpers.py:221: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))


### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [ ]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0035 sec, CPU: 0.00%, Memory: 5.61MB
Function `df_itertuple` executed in 0.0036 sec, CPU: 0.00%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [ ]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0366 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [ ]:
prev_month_blobs.head(1)

,geometry,id,geohash
8507,"POLYGON ((-98.16996 26.92231, -98.16996 26.922...",e8ffb175-5a75-44ab-abb0-b1b251411dcb,9uf1nj55


In [ ]:
prev_month_blobs_wkt[0]

('POLYGON ((-98.169962 26.922313, -98.169962 26.922904, -98.169254 26.922904, -98.169254 26.922313, -98.169962 26.922313))',
 'e8ffb175-5a75-44ab-abb0-b1b251411dcb',
 '9uf1nj55')

In [ ]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 10000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.3979 sec, CPU: 0.00%, Memory: 0.00MB
Table curr_blobs_wkt truncated.
Inserted 10000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.3672 sec, CPU: 29.20%, Memory: 4.58MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [ ]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 10000 records from prev_blobs_wkt.
Retrieved 10000 records from curr_blobs_wkt.


Compare the tables before and after for a sanity check:

In [ ]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-98.169962 26.922313, -98.169962 26....",e8ffb175-5a75-44ab-abb0-b1b251411dcb,9uf1nj55
1,"POLYGON ((-98.246043 27.178699, -98.246043 27....",699be616-0f16-4aec-9658-8e1fb7c995db,9uf4s7g8
2,"POLYGON ((-98.258909 27.304334, -98.258909 27....",4626d0ee-a7c1-4f8f-8f0f-0c2b351e59c2,9uf5k4dn
3,"POLYGON ((-97.901237 27.653225, -97.901237 27....",7849f3ec-fb7d-4caa-bae2-517c089283a5,9ufmk4nt
4,"POLYGON ((-98.217326 27.676151, -98.217326 27....",4708cb3c-6db9-4f7f-bb12-1491c09fcbe1,9ufjmn2d


In [ ]:
prev_month_blobs.head()

,geometry,id,geohash
8507,"POLYGON ((-98.16996 26.92231, -98.16996 26.922...",e8ffb175-5a75-44ab-abb0-b1b251411dcb,9uf1nj55
9127,"POLYGON ((-98.24604 27.17870, -98.24604 27.179...",699be616-0f16-4aec-9658-8e1fb7c995db,9uf4s7g8
1665,"POLYGON ((-98.25891 27.30433, -98.25891 27.305...",4626d0ee-a7c1-4f8f-8f0f-0c2b351e59c2,9uf5k4dn
3025,"POLYGON ((-97.90124 27.65322, -97.90124 27.653...",7849f3ec-fb7d-4caa-bae2-517c089283a5,9ufmk4nt
1518,"POLYGON ((-98.21733 27.67615, -98.21733 27.676...",4708cb3c-6db9-4f7f-bb12-1491c09fcbe1,9ufjmn2d


In [ ]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 10000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


### 2. Matching

We will match the polygons using GeoPandas.

In [ ]:
# postgresql_details = h.pg_details()
# h.run_parallel_matching(
#     "prev_blobs_wkt", "curr_blobs_wkt", "matched_results",
#     postgresql_details, "blob_matching", num_workers=4, batch_size=50
# )

In [ ]:
# postgresql_details = h.pg_details()
# h.run_parallel_matching(table_prev='prev_blobs_wkt', 
#                         table_curr='curr_blobs_wkt', 
#                         output_table='matched_results', 
#                         postgresql_details=postgresql_details, 
#                         db_name='blob_matching', 
#                         num_workers=4, 
#                         batch_size=100)

Baseline - Polygon Matching Only

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.